<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2023/ex06notebookB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex06notebookB

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2023)


----
## ニューラルネットワークと深層学習 (3) + 汎化と過適合 (2)
----





----
### 準備

以下，コードセルを上から順に実行してながら読んでいってね．

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import seaborn
seaborn.set()

# 機械学習ライブラリ scikit-learn のほげ
from sklearn.datasets import make_moons
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

---
### 階層型ニューラルネットワークによる識別

前の notebook では，階層型ニューラルネットワークの典型的な学習の方法について説明しました．
中間層が一つで損失関数が二乗誤差の場合について，ネットワークの入出力や学習アルゴリズムを具体的に式で表し，非線形回帰の問題への適用例を示しました．
ここでは，別の例として，識別問題への適用を考えます．



階層型ニューラルネットを識別問題に適用する場合，出力層のニューロンの活性化関数に softmax関数を用い，損失関数に交差エントロピーを用いるのが一般的です（注）．

<span style="font-size: 75%">
※ 注: 「softmax関数」や「交差エントロピー」は，「ロジスティック回帰＋勾配法によるパラメータの最適化 (3)」で登場しました．
</span>

#### 例: 中間層が一つで損失関数が交差エントロピーの場合

<img width="40%" src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/neuralnet6.png" align="right">

ここでは，図のように，中間層を一つもち，入力が$D$次元，中間層および出力層のニューロン数がそれぞれ $H$，$K$ のニューラルネットワークを考えます．$K$クラスの識別問題が対象で，出力層の $k$ 番目のニューロンの出力 $z_k$ は，入力データが $k$ 番目のクラスのものである確率を表します．

このネットワークの出力 $z_k$ は次式で表わされます．

$$
\begin{aligned}
y_h &= \sigma\left( v_{h,0} + \sum_{d=1}^{D}v_{h,d}x_d\right) & (h = 1, 2, \ldots, H) \\
z_k &= \frac{\displaystyle\exp\left(w_{k,0} + \sum_{h=1}^{H}w_{k,h}y_h \right)}{\displaystyle\sum_{m=1}^{K}\exp\left(w_{m,0} + \sum_{h=1}^{H}w_{m,h}y_h \right)} & (k=1,2,\ldots,K) 
\end{aligned}
$$

中間層ニューロンの活性化関数（式の中の $\sigma(\cdot)$）としては，ReLU関数やシグモイド関数などが用いられます（注）．また，出力層の活性化関数は，式が示す通り softmax 関数です．

<span style="font-size: 75%">
※注:「ニューラルネットワークと深層学習 (1)」参照．
</span>

学習データが $N$ 個与えられるとして，出力の正解を $\widetilde{z}_{n,k}$ ($n=1,2,\ldots,N$) とおくとき，損失関数 $L$ は，出力とその正解の間の交差エントロピー

$$
L = -\sum_{n=1}^{N}\sum_{k=1}^{K}\widetilde{z}_{n,k}\log{z_{n,k}}
$$

とします．以下，導出は省略しますが，前の notebook の二乗誤差損失関数の場合と同様にして，パラメータ $v_{h,d}, w_{k,h}$ に関する $L$ の勾配を求めることができます．


---
### 具体例: 2次元データの識別



#### 問題設定

2次元のデータを2クラスに分ける識別問題の例で，実際にニューラルネットを学習させる実験を行ってみましょう．

In [ ]:
# データの生成
NL, NT = 200, 2000
Xraw, Yraw = make_moons(n_samples=NL+NT, noise=0.3, random_state=2929)
XL, YL = Xraw[:NL], Yraw[:NL] # 学習データ  NL 個
XT, YT = Xraw[NL:], Yraw[NL:] # テストデータ NT 個

# 識別境界描画用データ
xmin, xmax = -2, 3
ymin, ymax = -1.5, 2
xx, yy = np.mgrid[xmin:xmax:0.02, ymin:ymax:0.02]
XX = np.dstack((xx, yy))

# データの散布図
fig, ax = plt.subplots(1, 2, figsize=(16, 8))
for i, s in enumerate(['Learning Data', 'Test Data']):
    if i == 0:
        X0, X1 = XL[YL == 0, :], XL[YL == 1, :]
    else:
        X0, X1 = XT[YT == 0, :], XT[YT == 1, :]
    ax[i].scatter(X0[:, 0], X0[:, 1], label='Class0', marker='.')
    ax[i].scatter(X1[:, 0], X1[:, 1], label='Class1', marker='.')
    ax[i].set_xlim(xmin, xmax)
    ax[i].set_ylim(ymin, ymax)
    ax[i].set_aspect('equal')
    ax[i].legend()
    ax[i].set_title(s)
plt.show()

上図の左が学習データ，右がテストデータの散布図です．

#### 実験

上記のデータをニューラルネットに学習させてみましょう．
ネットワークの構造は，上で説明した中間層が一つのもので，中間層のニューロン数を $H = 1000$ とします．また，比較のためにロジスティック回帰もやってみます．

ここでは，[scikit-learn](https://www.scikit-learn.org/) という機械学習ライブラリ用いています．

In [ ]:
#@title #### 2次元データの識別
#@markdown `useNN` が `True`（チェックがついている） だとニューラルネット，
#@markdown `False`だとロジスティック回帰

useNN = True #@param {type: 'boolean'}

# モデルの準備
if useNN:
    model = MLPClassifier(hidden_layer_sizes=(1000,), activation='relu', max_iter=500, alpha=0.0)
else:
    model = LogisticRegression()

# 学習
model.fit(XL, YL)
Yt = model.predict(XL)
ncL = np.sum(Yt == YL) 

# テスト
Yt = model.predict(XT)
ncT = np.sum(Yt == YT)
P = model.predict_proba(XX.reshape((-1, 2)))

# グラフの描画
fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(XL[YL == 0, 0], XL[YL == 0, 1], label='Class0')
ax.scatter(XL[YL == 1, 0], XL[YL == 1, 1], label='Class1')
pp = P[:, 0].reshape((XX.shape[0], XX.shape[1]))
ax.contourf(xx, yy, 1 - pp, cmap='bwr', alpha=0.2)
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_aspect('equal')
ax.legend()
plt.show()

# 識別率の出力
print(f'学習データの識別率: {ncL}/{NL} = {ncL/NL}')
print(f'テストデータの識別率: {ncT}/{NT} = {ncT/NT}')


図の色が塗られた領域は，ロジスティック回帰またはニューラルネットが予測した2つのクラスの確率を可視化しています．青色は `Class0` の確率が高いことを，赤色は `Class1` の確率が高いことを示します．

ロジスティック回帰では，モデルが予測する2クラスの境界（決定境界）が直線にしかなりませんが，ニューラルネットの場合，中間層によって非線形な変換が行えるおかげで，クラスをよりうまく識別できる複雑な境界を作ることができています．
その結果，識別率（識別の正解率）は，学習データについてもテストデータについても，ニューラルネットの方が高くなっています．

---
### 汎化と過適合(2)




<img width="40%" src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/modeldof.png" align="right">


「汎化と過適合」の回に，多項式当てはめの例を挙げて，次のことを説明しました：
- 教師あり学習においては，学習データに対して正解できるだけでなく，学習時に見たことのない未知データに対して **汎化** できることが重要
- データ数と比較してパラメータ数が多い複雑な（自由度が過剰な）モデルは汎化性能が低下する **過適合** を起こすことがある

階層型ニューラルネットの場合，中間層の数やニューロンの数によってパラメータの数が決まりますので，それらをいい加減に決めてしまうと過適合が起こり得ます．

試しに，上の具体例のデータを識別するニューラルネットの構造を変えて実験してみましょう．ここでは，次のような条件を考えます．

- 条件0: 中間層1層でニューロン数1000（上の実験で用いたのと同じ），パラメータ数は約4千（注）
- 条件1: 中間層2層でニューロン数はそれぞれ1000，1000，パラメータ数は約100万

<span style="font-size: 75%">
※ 注: $v_{h,d}$ が $h = 1, 2, \ldots, H, d = 0, 1, \ldots, D$ で $H(D+1)$ 個，$w_{k,h}$ が $k = 1, 2, \ldots, K, h = 0, 2, \ldots, H$ で $K(H+1)$個の合計 $H(D+1)+K(H+1)$ 個．
</span>

In [ ]:
# モデルの準備

#@title #### 汎化と過適合
#@markdown 0: 中間層1層でニューロン数1000

#@markdown 1: 中間層2層でニューロン数1000,1000

#@markdown 2: 中間層3層でニューロン数100,100,100


modelType = 1 #@param [0, 1, 2] {type: 'raw'}

archList = [
    (1000,),
    (1000, 1000),
    (100, 100, 100),
]
arch = archList[modelType]

# モデルの準備
model = MLPClassifier(hidden_layer_sizes=arch, activation='relu', max_iter=500, alpha=0.0)

# 学習
model.fit(XL, YL)
Yt = model.predict(XL)
ncL = np.sum(Yt == YL) 

# テスト
Yt = model.predict(XT)
ncT = np.sum(Yt == YT)
P = model.predict_proba(XX.reshape((-1, 2)))

# グラフの描画
fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(XL[YL == 0, 0], XL[YL == 0, 1], label='Class0')
ax.scatter(XL[YL == 1, 0], XL[YL == 1, 1], label='Class1')
pp = P[:, 0].reshape((XX.shape[0], XX.shape[1]))
ax.contourf(xx, yy, 1 - pp, cmap='bwr', alpha=0.2)
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_aspect('equal')
ax.legend()
plt.show()

# 識別率の出力
print(f'モデルの構成: {arch}')
print(f'学習データの識別率: {ncL}/{NL} = {ncL/NL}')
print(f'テストデータの識別率: {ncT}/{NT} = {ncT/NT}')


条件0と条件1の結果を比較すると，より複雑なモデルを用いる条件1の方が得られる境界が複雑で，学習データの識別率が高くなっています．しかし，テストデータの識別率は条件1の方が低くなっており，過適合が疑われる結果となっています（注）．

<span style="font-size: 75%">
※注: 階層型ニューラルネットの学習では最適解が得られる保証がありませんので，パラメータの初期値によってたまたま良い解に到達できなかったという場合もあり得ます．上のようにそれぞれの条件で1回ずつだけの実験ではその辺りの調査が不十分です．本当は，それぞれの条件でパラメータの初期値を何通りか変えて複数回学習させた結果を考察すべきところです．
</span>
